In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


In [2]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1753778719, 'localtime': '2025-07-29 01:45'}, 'current': {'last_updated_epoch': 1753778700, 'last_updated': '2025-07-29 01:45', 'temp_c': 15.4, 'temp_f': 59.7, 'is_day': 0, 'condition': {'text': 'Fog', 'icon': '//cdn.weatherapi.com/weather/64x64/night/248.png', 'code': 1135}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 238, 'wind_dir': 'WSW', 'pressure_mb': 1019.0, 'pressure_in': 30.08, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 87, 'cloud': 25, 'feelslike_c': 15.4, 'feelslike_f': 59.7, 'windchill_c': 12.5, 'windchill_f': 54.4, 'heatindex_c': 13.3, 'heatindex_f': 56.0, 'dewpoint_c': 12.8, 'dewpoint

In [3]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [4]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hi there! How can I help you today?'

In [5]:
model_with_tools = model.bind_tools(tools)

In [6]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I help you today?

Tool calls: []


In [7]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in SF'}, 'id': '6c8b3532-b7e0-402d-8513-aebce59808eb', 'type': 'tool_call'}]


In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [9]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I help you today?


In [10]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (9bdc5dd2-dc9b-48c1-a62f-ef7c7a31ff7c)
 Call ID: 9bdc5dd2-dc9b-48c1-a62f-ef7c7a31ff7c
  Args:
    query: weather in SF
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in SF", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1753780818, 'localtime': '2025-07-29 02:20'}, 'current': {'last_updated_epoch': 1753780500, 'last_updated': '2025-07-29 02:15', 'temp_c': 14.4, 'temp_f': 57.9, 'is_day': 0, 'condition': {

In [11]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (89a3bffd-cfc3-4b3c-ae18-71288de071c8)
 Call ID: 89a3bffd-cfc3-4b3c-ae18-71288de071c8
  Args:
    query: weather in SF
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in SF", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1753780818, 'localtime': '2025-07-29 02:20'}, 'current': {'last_updated_epoch': 1753780500, 'last_updated': '2025-07-29 02:15', 'temp_c': 14.4, 'temp_f': 57.9, 'is_day': 0, 'condition': {

In [12]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The weather in San Francisco, California on Tuesday, July 29, 2025, is partly cloudy with a temperature of 57.9|°F (14.4°C). The wind is 7.4 mph from the WSW, and humidity is 90%. The day temperature is expected to be 63°F and the night temperature 55°F|, with 0% precipitation.|

In [13]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [14]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [15]:
input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hello Bob! How can I help you today?


In [16]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I'm sorry, I don't have memory of past conversations. I only know that you introduced yourself as Bob in the current conversation.


In [17]:
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I do not know your name. I am a large language model, able to communicate in response to a wide range of prompts and questions, but I have no memory of past interactions and do not store personal information.
